In [ ]:
import numpy as np
import xfab.tools
import os
import numpy as np
import matplotlib.pyplot as plt
from xfab.symmetry import Umis
import xfab.symmetry
from collections import deque
from scipy.ndimage import binary_fill_holes
from scipy.spatial.transform import Rotation
import matplotlib
import torch
import skimage
import scipy.ndimage

In [ ]:
keys = ['layer_top', 'layer_center', 'layer_bottom']
dirs = [os.path.join('..', dir, 'test_run') for dir in keys]
sample = {}
sample['layer_top']    = np.fliplr(  np.load(os.path.join(dirs[0], 'sample_mask_layer_top.npy'))[5:-5, 5:-5]  )
sample['layer_center'] = np.fliplr(  np.load(os.path.join(dirs[1], 'sample_mask_layer_center.npy'))[2:-2, 2:-2] )
sample['layer_bottom'] = np.fliplr(  np.load(os.path.join(dirs[2], 'sample_mask_layer_bottom.npy'))[2:-2, 2:-2] )

stress = {}
stress['layer_top'] = np.load('../layer_top/test_run/stress_map_top_layer_refined_com_corrected.npy')
stress['layer_center'] = np.load('../layer_center/test_run/stress_map_central_layer_refined_com_corrected.npy')
stress['layer_bottom'] = np.load('../layer_bottom/test_run/stress_map_bottom_layer_refined_com_corrected.npy')

strain = {}
strain['layer_top'] = np.load('../layer_top/test_run/strain_map_top_layer_refined_com_corrected.npy')
strain['layer_center'] = np.load('../layer_center/test_run/strain_map_central_layer_refined_com_corrected.npy')
strain['layer_bottom'] = np.load('../layer_bottom/test_run/strain_map_bottom_layer_refined_com_corrected.npy')


ubi_map, X, Y, ipf_map, xypf_map = {},{},{},{}, {}
for i,key in enumerate(keys):
    X[key] = np.load( os.path.join( dirs[i], 'X_coord_gmap.npy' ) )
    Y[key] = np.load( os.path.join( dirs[i], 'Y_coord_gmap.npy' ) )
    ubi_map[key]    = np.load(os.path.join(dirs[i], 'ubi_median_filt_s6.npy'))
    ipf_map[key]    = np.load(os.path.join(dirs[i], 'ipf_median_filt_s6.npy'))
    xypf_map[key]    = np.load(os.path.join(dirs[i], 'xypf_median_filt_s6.npy'))

n = np.max( stress['layer_top'].shape + stress['layer_center'].shape + stress['layer_bottom'].shape )
stress_3D = np.zeros( (3, n,n,3,3) )
for i,key in enumerate(keys):
    k = stress[key].shape[0]
    pad = (n-k)//2
    if pad==0: stress_3D[i] = stress[key]
    else:stress_3D[ i, pad:-pad, pad:-pad] = stress[key]

In [ ]:
for key in keys: 
    dd = sample[key].sum(axis=0)
    ee = sample[key].sum(axis=1)
    print( 'mean side length : ', np.sqrt(sample[key].sum()) )


In [ ]:
grain_boundaries = {}
for i,key in enumerate(keys):
    grain_boundaries[key] = np.load(key + '_grain_boundaries.npy')

In [ ]:
plt.figure(figsize=(9,4))
b = None
for i,key in enumerate(keys):
    s = stress[key][:,:,2,2] / 1e6 # Pa to MPa
    s = s[sample[key]]    
    if b is None: 
        bins = np.linspace(-100,250,50)
        h,b = np.histogram(s, bins=bins, density=True)
        bc = ( b[0:-1] + b[1:] ) /2.
    else:
        h,_ = np.histogram(s, bins=b, density=True)
    dx = bc[1]-bc[0]
    plt.plot(bc, h, marker='o ^ s'.split(' ')[i], linestyle='--', linewidth=1, label=key.replace('_', ' '), markersize=4)
    A = 1.5*1.5*1e-6*1e-6 
    XSA = A*np.sum(mask)
    F = np.sum( stress[key][:,:,2,2] * A )
    S =  F / XSA
    print('\n '+ key)
    print( 'X-Area [mum^2] : ',  XSA*1e6*1e6)
    print( 'Sum zz-Stress / area [N] : ' , stress[key][sample[key],2,2].sum()* A )   
    print( 'Mean zz-Stress [MPa] : ' , stress[key][sample[key],2,2].mean() / 1e6)   

    print(strain[key][:,:,2,2].max()*1e4)

plt.grid()
plt.legend()
plt.xlabel('$\sigma_{zz}$  [MPa]')
plt.ylabel('Fractional Cross Section Area  [-]')
plt.show()


In [ ]:
plt.figure(figsize=(9,4))

colors = np.array( [
    [55,  126, 184],  #377eb8 
    [255, 127, 0],    #ff7f00
    [77,  175, 74],   #4daf4a
    [247, 129, 191],  #f781bf
    [166, 86,  40],   #a65628
    [152, 78,  163],  #984ea3
    [153, 153, 153],  #999999
    [228, 26,  28],   #e41a1c
    [222, 222, 0]     #dede00
] )
colors = colors / np.max(colors, axis=1).reshape(9,1)

elim  = (85*1e6) / (70*1e9)  # proof stress / E-modulus, https://www.azom.com/article.aspx?ArticleID=2798
v = 0.33

marker = ['^', 'o', 's', '*', 'd', '+']
sxx = 1e3*np.concatenate( [ strain[key][:,:,0,0].copy()[sample[key]] for key in keys ] )
syy = 1e3*np.concatenate( [ strain[key][:,:,1,1].copy()[sample[key]] for key in keys ] )
szz = 1e3*np.concatenate( [ strain[key][:,:,2,2].copy()[sample[key]] for key in keys ] )
sxy = 1e3*np.concatenate( [ strain[key][:,:,0,1].copy()[sample[key]] for key in keys ] )
sxz = 1e3*np.concatenate( [ strain[key][:,:,0,2].copy()[sample[key]] for key in keys ] )
syz = 1e3*np.concatenate( [ strain[key][:,:,1,2].copy()[sample[key]] for key in keys ] )


b = np.arange(-1.5, 2.2, 0.1)
h,b = np.histogram(szz, bins=b, density=False)
bc = ( b[0:-1] + b[1:] ) /2.

plt.plot(bc, h / np.sum(h), marker=marker[0], linestyle='--', color=colors[0], label='$\epsilon_{zz}$ ')

h,b = np.histogram(sxx, bins=b, density=False)
plt.plot(bc, h / np.sum(h), marker=marker[1], linestyle='--', color=colors[1], label='$\epsilon_{xx}$ ')

h,b = np.histogram(syy, bins=b, density=False)
plt.plot(bc, h/ np.sum(h), marker=marker[2], linestyle='--', color=colors[2], label='$\epsilon_{yy}$ ')

h,b = np.histogram(sxy, bins=b, density=False)
plt.plot(bc, h/ np.sum(h), marker=marker[3], linestyle='--', color=colors[3], label='$\epsilon_{xy}$ ')

h,b = np.histogram(sxz, bins=b, density=False)
plt.plot(bc, h/ np.sum(h), marker=marker[4], linestyle='--', color=colors[8], label='$\epsilon_{xz}$ ')

h,b = np.histogram(syz, bins=b, density=False)
plt.plot(bc, h/ np.sum(h), marker=marker[5], linestyle='--', color=colors[7], label='$\epsilon_{yz}$ ')


print('Tabulated Elastic limit : ', elim*1e3)
print('Fraction abbove elastic limit:', (np.sum(np.abs(szz) > elim*1e3))/len(szz) )
print( 'Mean z strain : ', szz.mean() )
print( 'Mean x strain : ', sxx.mean() )
print( 'Mean y strain : ', syy.mean() )
print( 'Tabulated Poisson Ratio : ', v)
print( 'Measured Poisson Ratio xx: ', (-sxx.mean() / szz.mean()))
print( 'Measured Poisson Ratio yy: ', (-syy.mean() / szz.mean()))

plt.grid()
plt.legend(fontsize=16)
plt.xlabel('Strain [x 1e-3]')
plt.ylabel('Volume Fraction [-]')

plt.savefig('gallery/strain_distributions.png', dpi=400, pad_inches=0.000001, bbox_inches='tight')

plt.show()


In [ ]:

components = [[0,0],[1,1],[2,2],[0,1],[0,2],[1,2]]
titles= 'xx yy zz xy xz yz'.split(' ')
for key in keys:

    for component,title in zip(components, titles):
        fig, ax = plt.subplots(1,1,figsize=(9,9))
        coords = np.argwhere(sample[key])
        x_min, y_min = coords.min(axis=0)
        x_max, y_max = coords.max(axis=0)

        gb = grain_boundaries[key][x_min:x_max+1, y_min:y_max+1].copy()

        cropped = stress[key][x_min:x_max+1, y_min:y_max+1].copy()

        Xc = X[key][x_min:x_max+1, y_min:y_max+1]
        Yc = Y[key][x_min:x_max+1, y_min:y_max+1]

        cmap = matplotlib.colormaps.get_cmap('RdBu_r')

        mask = sample[key].copy()[x_min:x_max+1, y_min:y_max+1]
        rgb_arr = cropped[:,:,component[0],component[1]] / 1e6
        
        if title=='xx' or title=='yy' or title=='zz':
            vmax = 170
        else:
            vmax = 40

        vmin = -vmax 

        print(np.max(np.abs(stress_3D[:,:,component[0],component[1]])) )

        norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)

        rgb_image = cmap( norm(rgb_arr) )
        rgb_image[gb, 0:3] = 0

        rgb_image[~mask, -1] = 0

        im = ax.pcolormesh( Xc, Yc, rgb_image )

        cax = fig.add_axes([0.95, 0.14, 0.035, 0.7])

        bar = np.linspace(vmax, vmin, 500)
        cax.imshow( bar.reshape(500,1) * np.ones( (500, 10) ), cmap='RdBu_r' )

        positions = np.linspace(0, len(bar)-1, 9).astype(int)
        labels = [np.round(bar[i]).astype(int) for i in positions]

        cax.set_yticks(positions, labels)
        cax.set_xticks([], [])
        cax.yaxis.tick_right() 


        cax.spines[['right', 'top', 'left', 'bottom']].set_visible(False)
        cax.tick_params(axis='both', which='major', labelsize=12)

        ax.axis('off')

        ax.annotate( '[MPa]', (Xc[-1,-1] ,  Yc[-10,-10]), size=16 )


        ax.set_title(key.replace('_', ' ') + ' - $\sigma_{'+title+'}$', fontsize=24)
        # ax.spines[['right', 'top', 'left', 'bottom']].set_visible(False)
        # ax.set_ylabel('x [$\\mu$m]', size=16)
        # ax.set_xlabel('y [$\\mu$m]', size=16)
        # ax.tick_params(axis='both', which='major', labelsize=16)
        # ax.tick_params(axis='both', which='minor', labelsize=16)

        #ax.axis('equal')

        fig.savefig('gallery/' + key +'_stress_'+title+'.png', dpi=400, pad_inches=0.000001, bbox_inches='tight')
        plt.show()

In [ ]:
hydrostatic, devatoric, effective, J2, J3, triaxial, L, theta = {}, {}, {}, {}, {}, {}, {}, {}
for key in keys:
    hydrostatic[key] = ( stress[key][:,:,0,0] + stress[key][:,:,1,1] + stress[key][:,:,2,2] ) / 3.
    devatoric[key] = stress[key].copy()
    devatoric[key][:,:,0,0]   -=  hydrostatic[key]
    devatoric[key][:,:,1,1]   -=  hydrostatic[key]
    devatoric[key][:,:,2,2]   -=  hydrostatic[key]
    J2[key]    = ( np.sum(devatoric[key]*devatoric[key], axis=(-2,-1)) )/2.
    effective[key] = np.sqrt(3*J2[key]) 

    J3[key] = np.zeros_like(J2[key])
    for i in range(stress[key].shape[0]):
        for j in range(stress[key].shape[1]):
            J3[key][i,j] = np.linalg.det(devatoric[key][i,j])

    denom = (effective[key] + 1e-32).copy()
    triaxial[key] = ( hydrostatic[key] / denom ) * (effective[key]!=0)
    L[key] = (effective[key]!=0) * (-27/2.)*J3[key]/( denom**3 )
    theta[key] = np.degrees( np.arccos(-L[key]) / 3. )

In [ ]:
above_uts_stress_effective = 0
above_uts_stress_zz = 0 
total_voxels = 0
for key in keys:
    above_uts_stress_effective += np.sum(( effective[key][sample[key]] / 1e6) > 85)
    above_uts_stress_zz += np.sum( ( stress[key][sample[key],2,2] / 1e6) > 85 )
    total_voxels += np.sum( sample[key] )
print('total number of voxels in volume (z1+z2+z3) : ', total_voxels)
print('fraction of voxels above uts (85 MPA) - effective stress: ', above_uts_stress_effective / float(total_voxels) )
print('fraction of voxels above uts (85 MPA) - zz stress: ', above_uts_stress_zz / float(total_voxels))


In [ ]:
plt.hist(effective[keys[1]][sample[keys[1]]] / 1e6, bins=100)
plt.title('Histogram of effective Stress')
plt.legend()
plt.xlabel('Effective stress ($\sigma_h$/$\sigma_{eff}$) [-]')
plt.ylabel('Counts')
plt.show()

In [ ]:


components = [1,2,3,4]
titles= 'e m t \theta'.split(' ')
vv = [None, [0,150], [-150,150], [-1,2], [0,60] ]
for key in keys:

    for component,title in zip(components, titles):

        fig, ax = plt.subplots(1,1,figsize=(9,9))
        coords = np.argwhere(sample[key])
        x_min, y_min = coords.min(axis=0)
        x_max, y_max = coords.max(axis=0)

        gb = grain_boundaries[key][x_min:x_max+1, y_min:y_max+1].copy()

        if component==1:
            cropped = effective[key][x_min:x_max+1, y_min:y_max+1]
            rgb_arr = cropped / 1e6
            cmap = 'RdBu_r'
        if component==2:
            cropped = hydrostatic[key][x_min:x_max+1, y_min:y_max+1]
            rgb_arr = cropped / 1e6
            cmap = 'RdBu_r'
        if component==3:
            cropped = triaxial[key][x_min:x_max+1, y_min:y_max+1]
            rgb_arr = cropped
            cmap = 'RdBu_r'
        if component==4:
            cropped = theta[key][x_min:x_max+1, y_min:y_max+1]
            rgb_arr = cropped
            cmap = 'RdBu_r'


        Xc = X[key][x_min:x_max+1, y_min:y_max+1]
        Yc = Y[key][x_min:x_max+1, y_min:y_max+1]

        cmap = matplotlib.colormaps.get_cmap(cmap)


        mask = sample[key].copy()[x_min:x_max+1, y_min:y_max+1]
        
        vmax = vv[component][1]
        vmin = vv[component][0]
        norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)

        rgb_image = cmap( norm(rgb_arr) )
        rgb_image[gb, 0:3] = 0

        rgb_image[~mask, -1] = 0

        im = ax.pcolormesh( Xc, Yc, rgb_image )

        cax = fig.add_axes([0.95, 0.14, 0.035, 0.7])

        bar = np.linspace(vmax, vmin, 500)

        
        cax.imshow( bar.reshape(500,1) * np.ones( (500, 10) ), cmap )

        positions = np.linspace(0, len(bar)-1, 3).astype(int)
        labels = [np.round(bar[i]).astype(int) for i in positions]

        cax.set_yticks(positions, labels)
        cax.set_xticks([], [])
        cax.yaxis.tick_right() 


        cax.spines[['right', 'top', 'left', 'bottom']].set_visible(False)
        cax.tick_params(axis='both', which='major', labelsize=12)

        ax.axis('off')

        if component==1 or component==2: 
            ax.annotate( '[MPa]', (Xc[-1,-1] ,  Yc[-10,-10]), size=16 )
            ax.set_title(key.replace('_', ' ') + ' - $\sigma_{'+title+'}$', fontsize=24)
        if component==3: 
            ax.annotate( '[-]', (Xc[-1,-1] ,  Yc[-10,-10]), size=16 )
            ax.set_title(key.replace('_', ' ') + ' - $\sigma_{'+title+'}$', fontsize=24)
        if component==4: 
            ax.annotate( '[$^o$]', (Xc[-1,-1] ,  Yc[-10,-10]), size=16 )
            ax.set_title(key.replace('_', ' ') + ' - $\\theta$', fontsize=24)

        
        # ax.spines[['right', 'top', 'left', 'bottom']].set_visible(False)
        # ax.set_ylabel('x [$\\mu$m]', size=16)
        # ax.set_xlabel('y [$\\mu$m]', size=16)
        # ax.tick_params(axis='both', which='major', labelsize=16)
        # ax.tick_params(axis='both', which='minor', labelsize=16)

        #ax.axis('equal')

        print(cropped.max())

        fig.savefig('gallery/' + key +'_stress_'+title+'.png', dpi=400, pad_inches=0.000001, bbox_inches='tight')
        plt.show()

In [ ]:
def crystal_direction_cubic( ubi, axis ):
    hkl = np.dot( ubi, axis )
    # cubic symmetry implies:
    #      24 permutations of h,k,l
    #      one has abs(h) <= abs(k) <= abs(l)
    hkl = abs(hkl)
    hkl.sort()
    return hkl

def hkl_to_color_cubic( hkl ):
    """
    https://mathematica.stackexchange.com/questions/47492/how-to-create-an-inverse-pole-figure-color-map
        [x,y,z]=u⋅[0,0,1]+v⋅[0,1,1]+w⋅[1,1,1].
            These are:
                u=z−y, v=y−x, w=x
                This triple is used to assign each direction inside the standard triangle

    makeColor[{x_, y_, z_}] :=
         RGBColor @@ ({z - y, y - x, x}/Max@{z - y, y - x, x})
    """
    x,y,z = hkl
    assert x<=y<=z
    assert z>=0
    u,v,w = z-y, y-x, x
    m = max( u, v, w )
    r,g,b = u/m, v/m, w/m
    return (r,g,b)

def ubi_from_rgb_cubic( rgb, axes ):
    hkl = np.zeros_like(rgb)
    hkl[0,:] = rgb[2,:]
    hkl[1,:] = rgb[1,:] + hkl[0,:]
    hkl[2,:] = rgb[0,:] + hkl[1,:]
    # hkl = rgb @ axes
    ubi = ( hkl @ np.linalg.inv(axes) )
    if np.dot( np.cross(ubi.T[:,0], ubi.T[:,1]), ubi.T[:,2] ) < 0:
        ubi[:,2] *= -1
    return ubi

def hkl_to_pf_cubic( hkl ):
    x,y,z = hkl
    assert x<=y<=z
    assert z>=0
    m = np.sqrt((hkl**2).sum())
    return x/(z+m), y/(z+m)

def triangle(  ):
    """ compute a series of point on the edge of the triangle """
    xy = [ np.array(v) for v in ( (0,1,1), (0,0,1), (1,1,1) ) ]
    xy += [ xy[2]*(1-t) + xy[0]*t for t in np.linspace(0.1,1,15)]
    return np.array( [hkl_to_pf_cubic( np.array(p) ) for p in xy] )



In [ ]:
eff  = np.concatenate( [ effective[key][sample[key]] for key in keys ] )
ipf  = np.concatenate( [ ipf_map[key][sample[key]] for key in keys ] )
xypf = np.concatenate( [ xypf_map[key][sample[key]] for key in keys ] )
ubis = np.concatenate( [ ubi_map[key][sample[key]] for key in keys ] )
angle = []
zhat= np.array([0,0,1])
for ubi in ubis:
    #u = xfab.tools.ubi_to_u(ubi)
    cell = np.linalg.inv(ubi).T
    cell_axes = cell / np.linalg.norm(cell, axis=0)
    qq1 = cell_axes.T @ ( zhat )
    qq2 = cell_axes.T @ (-zhat )
    tt = np.arccos(np.concatenate((qq1,qq2)))
    angle.append( np.degrees( tt ).min() )
angle = np.array(angle)

In [ ]:
np.max(angle), np.min(angle)

In [ ]:

dzero_cell = [4.04, 4.04, 4.04, 90,90,90]
B0 = xfab.tools.form_b_mat(dzero_cell) / (np.pi*2)

# https://next-gen.materialsproject.org/materials/mp-134
D_Al = np.array( [
        [  104,   73, 	  73,    0,    0, 	 0   ],
        [  73, 	 104, 	  73, 	 0,    0, 	 0   ],
        [  73, 	  73, 	 104, 	 0,    0, 	 0   ],
        [  0, 	  0, 	  0, 	32,    0,    0   ],
        [  0, 	  0, 	  0, 	 0,    32, 	 0   ],
        [  0, 	  0, 	  0, 	 0,    0,    32  ]] ) # units of GPa     

C_c = np.linalg.inv(B0).T
E  = np.column_stack((C_c[:, 0], np.cross(C_c[:, 2], C_c[:, 0]), C_c[:, 2]))
E /= np.linalg.norm(E, axis=0)

def to_tensor( s ):
    e11, e22, e33, e12, e13, e23 = s
    return np.array([[e11,e12,e13],[e12,e22,e23],[e13,e23,e33]])

def to_list( s ):
    return np.array( [s[0,0], s[1,1], s[2,2], s[0,1], s[0,2], s[1,2]] )

def get_stress_Al_elastic( strain_sample_tensor, U ):
    strain_crystal_tensor =  E.T @( U.T @ strain_sample_tensor @ U ) @ E    
    e11, e22, e33, e12, e13, e23 = to_list( strain_crystal_tensor )
    strain_crystal_voigt = np.array( [e11, e22, e33, 2*e23, 2*e13, 2*e12] )
    stess_crystal_voigt = D_Al @ strain_crystal_voigt
    s11, s22, s33, s23, s13, s12 = stess_crystal_voigt
    stess_crystal = np.array( [s11, s22, s33, s12, s13, s23] )
    stess_crystal_tensor = to_tensor( stess_crystal )
    stess_sample_tensor = U @ ( E @ stess_crystal_tensor @ E.T ) @ U.T
    return stess_sample_tensor # note that the stress is now in GPa!



In [ ]:

sxx = np.concatenate([strain[key][sample[key], 0, 0] for key in keys])
syy = np.concatenate([strain[key][sample[key], 1, 1] for key in keys])
szz = np.concatenate([strain[key][sample[key], 2, 2] for key in keys])
sxy = np.concatenate([strain[key][sample[key], 0, 1] for key in keys])
sxz = np.concatenate([strain[key][sample[key], 0, 2] for key in keys])
syz = np.concatenate([strain[key][sample[key], 1, 2] for key in keys])

ss = np.vstack((sxx, syy, szz, sxy, sxz, syz))

covariance = np.cov( ss )
mean       = np.mean( ss, axis=1 )

draw = np.random.multivariate_normal( mean, covariance, size=(20000,) )




In [ ]:
us = np.array( [xfab.tools.ubi_to_u(ubis[i]) for i in range(ubis.shape[0])] )


In [ ]:
np.random.seed(1)
zhat = np.array([0,0,1])
perm = np.random.permutation(ss.shape[1])
ss_shuffle = ss.copy()[:, perm]


rand_angles, rand_data = [], []
for i in range(ubis.shape[0]):

    ubi = ubis[i]
    cell = np.linalg.inv(ubi).T
    cell_axes = cell / np.linalg.norm(cell, axis=0)
    qq1 = cell_axes.T @ ( zhat )
    qq2 = cell_axes.T @ (-zhat )
    tt = np.arccos(np.concatenate((qq1,qq2)))
    rand_angles.append( np.degrees( tt ).min() )

    U = us[i]#xfab.tools.ubi_to_u(ubi)
    #U = Rotation.random().as_matrix()

    #vec = draw[i]
    sxx, syy, szz, sxy, sxz, syz = ss_shuffle[:, i]
    _strain = np.array([[sxx, sxy, sxz], [sxy, syy, syz], [sxz, syz, szz]])

    #U = Rotation.random().as_matrix()
    #angle1 = np.degrees( np.arccos(U.T @  zhat) ).min()
    #angle2 = np.degrees( np.arccos(U.T @ -zhat) ).min()
    #rand_angles.append( np.min([angle1, angle2]) )

    _stress = get_stress_Al_elastic( _strain, U ) * 1e9 # Pa
    _dev = _stress - np.eye(3)*np.trace(_stress)/3.
    _eff = np.sqrt( 3 * np.sum( _dev*_dev ) / 2. ) / 1e6 # MPa

    rand_data.append( _eff )


In [ ]:
effective_strain = []
for s in ss.T:
    sxx, syy, szz, sxy, sxz, syz = s
    _strain = np.array([[sxx, sxy, sxz], [sxy, syy, syz], [sxz, syz, szz]])
    dev_strain = _strain - np.trace(_strain)*np.eye(3) / 3.
    effective_strain.append( np.sum( np.sqrt( 3*np.sum(dev_strain*dev_strain)/2 ) ) )
effective_strain = np.array(effective_strain)

In [ ]:
p_strain = np.polyfit(angle, effective_strain*1e4, deg=1)
print(p_strain)

In [ ]:
effective_strain.min()*1e4, effective_strain.max()*1e4, effective_strain.mean()*1e4

In [ ]:
x = np.linspace(0,60,120)

plt.figure(figsize=(12,7))

idx = np.random.permutation(len(angle))
plt.scatter(angle[idx], effective_strain[idx]*1e4, c=ipf[:,:,2][idx], s=0.3, label='Voxel Data')
plt.plot(x, np.polyval( p_strain, x ), '-k', label='Observed Trend')

plt.ylim([0, 35])
plt.xlim([0, 55])

plt.xlabel('Angle Between Unit Cell Face Normal and Tensile Load Axis [$^o$] ', fontsize=12)
plt.ylabel('Effective Strain $\sigma_e$ [x 1e4] ', fontsize=12)
#plt.legend()
plt.grid()

plt.savefig('gallery/strain_trend.svg', dpi=400, pad_inches=0.000001, bbox_inches='tight')
plt.savefig('gallery/strain_trend.png', dpi=400, pad_inches=0.000001, bbox_inches='tight')

plt.show()



In [ ]:
rand_p = np.polyfit(rand_angles, y=rand_data, deg=1 )
p_stress = np.polyfit(angle, eff /1e6, deg=1)
print('Random : ', rand_p)
print('Measured : ', p_stress)

In [ ]:
x = np.linspace(0,60,120)

plt.figure(figsize=(12,7))

idx = np.random.permutation(len(angle))
plt.scatter(angle[idx], eff[idx] / 1e6, c=ipf[:,:,2][idx], s=0.3, label='Voxel Data')
plt.plot(x, np.polyval( p_stress, x ), '-k', label='Observed Trend')
plt.plot(x, np.polyval( rand_p, x ), '--k', label='Random Expected Trend')



binsize = 2

nbins = int( (angle.max() - angle.min()) // binsize )

n, _ = np.histogram(angle, bins=nbins)
sy, _ = np.histogram(angle, bins=nbins, weights=eff)
sy2, bc = np.histogram(angle, bins=nbins, weights=eff*eff)
mean = sy / n
std = np.sqrt(sy2/n - mean*mean)

plt.errorbar((bc[1:] + bc[:-1])/2, mean/ 1e6, yerr=std / 1e6, fmt='k-',capsize=5)
plt.plot( (bc[1:] + bc[:-1])/2, mean / 1e6, 'ko--')


plt.ylim([15, 200])
plt.xlim([0, 55])

plt.xlabel('Angle Between Unit Cell Face Normal and Tensile Load Axis [$^o$] ', fontsize=12)
plt.ylabel('Effective Stress $\sigma_e$ [MPa] ', fontsize=12)
#plt.legend()
plt.grid()

plt.savefig('gallery/mises_trend.svg', dpi=400, pad_inches=0.000001, bbox_inches='tight')
plt.savefig('gallery/mises_trend.png', dpi=400, pad_inches=0.000001, bbox_inches='tight')

plt.show()



In [ ]:

binsize = 2

nbins = int( (angle.max() - angle.min()) // binsize )
print(nbins)
n, _ = np.histogram(angle, bins=nbins)
sy, _ = np.histogram(angle, bins=nbins, weights=eff)
sy2, bc = np.histogram(angle, bins=nbins, weights=eff*eff)
mean = sy / n
std = np.sqrt(sy2/n - mean*mean)

plt.errorbar((bc[1:] + bc[:-1])/2, mean/ 1e6, yerr=std / 1e6, fmt='k-',capsize=5)
plt.plot( (bc[1:] + bc[:-1])/2, mean / 1e6, 'ko--')

plt.show()




In [ ]:
effective_strain, deviatoric_strain, J2_strain, hydrostatic_strain = {},{},{},{}
for key in keys:
    hydrostatic_strain[key] = ( strain[key][:,:,0,0] + strain[key][:,:,1,1] + strain[key][:,:,2,2] ) / 3.
    deviatoric_strain[key] = strain[key].copy()
    deviatoric_strain[key][:,:,0,0]   -=  hydrostatic_strain[key]
    deviatoric_strain[key][:,:,1,1]   -=  hydrostatic_strain[key]
    deviatoric_strain[key][:,:,2,2]   -=  hydrostatic_strain[key]
    J2_strain[key]    = ( np.sum(deviatoric_strain[key]*deviatoric_strain[key], axis=(-2,-1)) )/2.
    effective_strain[key] = np.sqrt(3*J2_strain[key]) 
eff_strain  = np.concatenate( [ effective_strain[key][sample[key]] for key in keys ] )


In [ ]:

eff_strain  = np.concatenate( [ strain[key][:,:,2,2][sample[key]] for key in keys ] )

p = np.polyfit(angle, eff_strain * 1e4, deg=1)
x = np.linspace(0,60,120)
y = np.polyval( p, x )

plt.figure(figsize=(12,7))

idx = np.random.permutation(len(angle))
plt.scatter(angle[idx], eff_strain[idx] * 1e4, c=ipf[:,:,2][idx], s=0.3, label='Voxel Data')
plt.plot(x, y, '--k', label='Linear Trend')
plt.ylim([-2, 20])
plt.xlim([-1, 61])

plt.legend()
plt.grid()

plt.show()


In [ ]:
from scipy.stats import linregress
print(" STRESS ")
print(len(angle))
slope, intercept, r_value, p_value, std_err = linregress(angle, eff /1e6)
print( 'p_value',  p_value )
print( 'r_value',  p_value )
print( 'std_err',  std_err)
print( 'slope',    slope)
print(" \nSTRAIN ")
slope, intercept, r_value, p_value, std_err = linregress(angle, eff_strain * 1e4)
print( 'p_value',  p_value )
print( 'r_value',  p_value )
print( 'std_err',  std_err)
print( 'slope',    slope)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(7,7))
c = triangle().T
ax.axis('off')
ax.axis('equal')
ax.plot(c[0], c[1], 'k-', linewidth=3)
ax.scatter( xypf[:,0,2], xypf[:,1,2], c=ipf[:,:,2], s=2, linewidth=0)
fig.savefig('gallery/ipf_scatter.png', dpi=400, pad_inches=0.000001, bbox_inches='tight')

plt.show()

fig, ax = plt.subplots(1,1, figsize=(7,7))
ax.axis('equal')
ax.axis('off')
ax.plot(c[0], c[1], 'k-', linewidth=3)
im = ax.scatter( xypf[:,0,2], xypf[:,1,2], c = eff / 1e6, cmap='jet', linewidth=0, vmin=0, vmax=150, s=2, facecolor=None )
fig.colorbar(im, ax=ax)
fig.savefig('gallery/vmises_ipf_scatter.png', dpi=400, pad_inches=0.000001, bbox_inches='tight')
plt.show()
